# VIS/NIR Tutorial 

For dual VIS/NIR pipelines, a visible image is used to identify an image mask for the plant material.
The [get nir](get_nir.md) function is used to get the NIR image that matches the VIS image (must be in same folder,
with similar naming scheme), then functions are used to size and place the VIS image mask over the NIR image.
This allows two workflows to be done at once and also allows plant material to be identified in low-quality images.
We do not recommend this approach if there is a lot of plant movement between capture of NIR and VIS images.

To run a VIS/NIR pipeline over a single VIS image there are two required inputs:

1.  **Image:** Images can be processed regardless of what type of VIS camera was used (high-throughput platform, digital camera, cell phone camera).
Image processing will work with adjustments if images are well lit and free of background that is similar in color to plant material.  
2.  **Output directory:** If debug mode is set to 'print' output images from each intermediate step are produced.


In [ ]:
import sys, traceback
import cv2
import numpy as np
import argparse
import string
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/vis_nir/VIS_SV_image.jpg"
        self.debug = "plot"
        self.writeimg= False 
        self.result = "./vis_nir_tutorial_results"
        self.coresult = "./vis_nir_coresult"
        self.outdir = "."
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image (sometimes you need to run this line twice to see the image) 
img, path, filename = pcv.readimage(args.image)

In [ ]:
# Convert RGB to HSV and extract the saturation channel

# Inputs:
#   rgb_image - RGB image data 
#   channel - Split by 'h' (hue), 's' (saturation), or 'v' (value) channel
s = pcv.rgb2gray_hsv(img, 's')

In [ ]:
# Threshold the Saturation image

# Inputs:
#   gray_img - Grayscale image data 
#   threshold- Threshold value (between 0-255)
#   max_value - Value to apply above threshold (255 = white) 
#   object_type - 'light' (default) or 'dark'. If the object is lighter than the background then standard threshold is done.
#                 If the object is darker than the background then inverse thresholding is done. 
s_thresh = pcv.threshold.binary(s, 50, 255, 'light')

In [ ]:
# Median Blur

# Inputs: 
#   gray_img - Grayscale image data 
#   ksize - Kernel size (integer or tuple), (ksize, ksize) box if integer input,
#           (n, m) box if tuple input 
s_mblur = pcv.median_blur(s_thresh, 5)
s_cnt = pcv.median_blur(s_thresh, 5)

In [ ]:
# Convert RGB to LAB and extract the blue channel

# Input:
#   rgb_img - RGB image data 
#   channel- Split by 'l' (lightness), 'a' (green-magenta), or 'b' (blue-yellow) channel
b = pcv.rgb2gray_lab(img, 'b')

# Threshold the blue image
b_thresh = pcv.threshold.binary(b, 129, 255, 'light')

In [ ]:
# Join the thresholded saturation and blue-yellow images

# Inputs: 
#   bin_img1 - Binary image data to be compared to bin_img2
#   bin_img2 - Binary image data to be compared to bin_img1
bs = pcv.logical_and(s_mblur, b_thresh)

In [ ]:
# Apply Mask (for VIS images, mask_color=white)

# Inputs:
#   rgb_img - RGB image data 
#   mask - Binary mask image data 
#   mask_color - 'white' or 'black' 
masked = pcv.apply_mask(img, bs, 'white')

In [ ]:
# Identify objects

# Inputs: 
#   img - RGB or grayscale image data for plotting 
#   mask - Binary mask used for detecting contours 
id_objects,obj_hierarchy = pcv.find_objects(masked, bs)

In [ ]:
# Define ROI

# Inputs: 
#   x - The x-coordinate of the upper left corner of the rectangle 
#   y - The y-coordinate of the upper left corner of the rectangle 
#   h - The height of the rectangle 
#   w - The width of the rectangle 
#   img - RGB or grayscale image to plot the ROI on 
roi1, roi_hierarchy = pcv.roi.rectangle(800,450,920,700, img)

In [ ]:
# Decide which objects to keep

# Inputs:
#   img - RGB or grayscale image data to display kept objects on 
#   roi_type - 'cutto' or 'partial' => include objects that are partially inside or overlapping with the ROI 
#   roi_contour - contour of ROI, output from pcv.roi.rectangle in this case
#   object_contour - contour of objects, output from pcv.roi.rectangle in this case 
#   obj_hierarchy - heirarch of objects, output from pcv.find_objects function 
roi_objects, hierarchy, kept_mask, obj_area = pcv.roi_objects(img,'partial',roi1,roi_hierarchy,id_objects,obj_hierarchy)

In [ ]:
# Object combine kept objects

# Inputs:
#   img - RGB or grayscale image data for plotting 
#   contours - Contour list 
#   hierarchy - Contour hierarchy array 
obj, mask = pcv.object_composition(img, roi_objects, hierarchy)

In [ ]:
############### Analysis ################  
  
# Find shape properties, output shape image (optional) 

# Inputs:
#   img - RGB or grayscale image data 
#   obj- Single or grouped contour object
#   mask - Binary image mask to use as mask for moments analysis 
# Returns:
#   shape_header, shape_data, and analysis_images (an array containing the original image with shape
#   data plotted on it, and the mask) 
shape_header, shape_data, analysis_images = pcv.analyze_object(img, obj, mask)

In [ ]:
# Search for sharp angles 

# Inputs:
#   obj - A contour of the plant object 
#   window - The pre and post point distances on which to calculate 
#            the angle of the focal point on which to calculate 
#            the angle
#   thresh - Threshold to set for acuteness
#   sep - The number of contour points to search within for the 
#         most acute value 
#   img - RGB or grayscale image 
acute_points_list = pcv.acute_vertex(obj, 30, 25, 100, img)

In [ ]:
# You can automatically crop an image to a contour 

# Inputs: 
#   img - RGB or grayscale image data
#   object - Contour of target object 
#   padding_x - Padding in the x direction (default padding_x=0)
#   padding_y - Padding in the y direction (default padding_y=0)
#   color - Either 'black' (default) or 'white'
cropped_img = pcv.auto_crop(img, obj)

In [ ]:
# Shape properties relative to user boundary line 

# Inputs:
#   img - RGB or grayscale image data 
#   obj - Single or grouped contour object 
#   mask - Binary mask of selected contours 
#   line_position - Position of boundary line (a value of 0 would draw a line 
#                   through the bottom of the image) 
boundary_header, boundary_data, boundary_images_h = pcv.analyze_bound_horizontal(img, obj, mask, 700)

# Inputs:
#   img - RGB or grayscale image data 
#   obj - Single or grouped contour object 
#   mask - Binary mask of selected contours 
#   line_position - Position of boundary line (a value of 0 would draw a line 
#                   through the left of the image) 
boundary_header, boundary_data, boundary_images_v = pcv.analyze_bound_vertical(img, obj, mask, 1200)

In [ ]:
# Determine color properties: Histograms, Color Slices and Pseudocolored Images, output color analyzed images (optional)

# Inputs:
#   rgb_img - RGB image data
#   mask - Binary mask of selected contours 
#   bins - Number of color bins (0-256)
#   hist_plot_type - None (default), 'all', 'rgb', 'lab', or 'hsv'
 
color_header, color_data, color_histogram = pcv.analyze_color(img, kept_mask, 256, 'all')

In [ ]:
# Write shape and color data to results file
pcv.print_results(filename=args.result)

The next steps are to get the matching NIR image, resize, and place the VIS mask over it. 

In [ ]:
# coresult is set within the second jupyter line of code
if args.coresult is not None:
    nirpath = pcv.get_nir(path,filename)
    nir, path1, filename1 = pcv.readimage(nirpath)
    nir2 = cv2.imread(nirpath,0)

# Inputs:
#   img - RGB or grayscale image to resize
#   resize_x - resize number in the x dimension
#   resize_y - resize number in the y dimension 
nmask = pcv.resize(mask, 0.28,0.28)

# Inputs:
#   img - RGB or grayscale image data 
#   mask - Binary image to be used as a mask 
#   x - Amount to push in the vertical direction
#   y - Amount to push in the horizontal direction
#   v_pos - Push from the 'top' (default) or 'bottom' in the vertical direction
#   h_pos - Push from the 'right' (default) or 'left' in the horizontal direction 
newmask = pcv.crop_position_mask(nir, nmask, 34, 7, "top", "right")

In [ ]:
# Find objects in the new mask 

nir_objects, nir_hierarchy = pcv.find_objects(nir, newmask)

In [ ]:
# Combine objects

nir_combined, nir_combinedmask = pcv.object_composition(nir, nir_objects, nir_hierarchy)

In [ ]:
# Analyze the NIR intensity 

# Inputs: 
#   gray_img - 8 or 16-bit grayscale image data 
#   mask - Binary mask made from selected contours 
#   bins - Number of classes to divide spectrum into
#   histplot - If True then plots histogram of intensity values, (default False) 
nhist_header, nhist_data, nir_hist = pcv.analyze_nir_intensity(nir2, nir_combinedmask, 256, histplot=True)

# Plot the NIR histogram to the screen (since the hist is made with plotnine ggplot, no need to use pcv.plot_image) 
nir_hist

In [ ]:
# Analyze the shape of the object 
nshape_header, nshape_data, nir_images = pcv.analyze_object(nir2, nir_combined, nir_combinedmask)

In [ ]:
# The print_results function will take the measurements stored when running any (or all) of these functions, format, 
# and print an output text file for data analysis. The Outputs class stores data whenever any of the following functions
# are ran: analyze_bound_horizontal, analyze_bound_vertical, analyze_color, analyze_nir_intensity, analyze_object, 
# fluor_fvfm, report_size_marker_area, watershed. If no functions have been run, it will print an empty text file 
pcv.print_results(filename='vis_nir_tutorial_results.txt')